# find_images_to_be_fixed

Code was provided by the Voxel51 Docs AI in reponse to this prompt:

**Write a python script which uses voxel51 to identify images which need re-annotating and writes a report which includes filepath and problems detected for each image.**

In [1]:
ROOT_DIR = '/home/aubrey/Desktop/crb-damage-detection-models'
DATASET_DIR = f'{ROOT_DIR}/datasets/CRB005'
MODEL_DIR = f'{ROOT_DIR}/models/CRB005/weights/CRB005.pt'
FO_DATASET = 'fo_test_dataset_2'
OUTPUT_DIR = '/home/aubrey/Desktop/reannotation_dataset'

In [2]:
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F
import json
import shutil
import os
import pandas as pd

In [3]:
def wake_mongo():

    # List all local datasets by name

    # Reports error 26 on first try, indicating a problem opening the Mongo database.
    # The following hack runs the code a second time. Seems to work well.
    # Note: A dataset can be deleted using fo.delete_dataset("dataset name")
    try:
        dataset_list = fo.list_datasets()
    except Exception as e:
        print(f'ERROR: {e}')
        print("Retrying...")
        dataset_list = fo.list_datasets()
    print(f'Local FiftyOne datasets: {dataset_list}')
    
wake_mongo()

Subprocess ['/home/aubrey/Desktop/fifty_one/.venv/lib/python3.12/site-packages/fiftyone/db/bin/mongod', '--dbpath', '/home/aubrey/.fiftyone/var/lib/mongo', '--logpath', '/home/aubrey/.fiftyone/var/lib/mongo/log/mongo.log', '--port', '0', '--nounixsocket'] exited with error 62:


{"t":{"$date":"2025-11-01T06:30:29.545Z"},"s":"I",  "c":"CONTROL",  "id":20697,   "ctx":"main","msg":"Renamed existing log file","attr":{"oldLogPath":"/home/aubrey/.fiftyone/var/lib/mongo/log/mongo.log","newLogPath":"/home/aubrey/.fiftyone/var/lib/mongo/log/mongo.log.2025-11-01T06-30-29"}}
ERROR: fiftyone.core.service.DatabaseService failed to bind to port
Retrying...
Local FiftyOne datasets: ['fo_test_dataset', 'fo_test_dataset_2', 'quickstart']


In [4]:
# Load your dataset (replace with your actual dataset)
# dataset = foz.load_zoo_dataset("quickstart")
dataset = fo.load_dataset(FO_DATASET)
print(dataset)

Name:        fo_test_dataset_2
Media type:  image
Num samples: 151
Persistent:  True
Tags:        []
Sample fields:
    id:                fiftyone.core.fields.ObjectIdField
    filepath:          fiftyone.core.fields.StringField
    tags:              fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:          fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:        fiftyone.core.fields.DateTimeField
    last_modified_at:  fiftyone.core.fields.DateTimeField
    ground_truth:      fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    predictions:       fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    mistakenness:      fiftyone.core.fields.FloatField
    possible_missing:  fiftyone.core.fields.IntField
    possible_spurious: fiftyone.core.fields.IntField
    eval_tp:           fiftyone.core.fields.IntField
    eval_fp:           fiftyone.core.fields.IntFie

In [ ]:
# Ensure you have model predictions on your dataset
# If not, apply a model first:
# model = foz.load_zoo_model("your-model")
# dataset.apply_model(model, "predictions")

# Evaluate detections to identify potential issues
results = dataset.evaluate_detections(
    "predictions",
    gt_field="ground_truth",
    eval_key="eval",
    compute_mAP=True,
)

# Compute mistakenness to find annotation errors
import fiftyone.brain as fob
fob.compute_mistakenness(
    dataset,
    "predictions",
    label_field="ground_truth",
)

# Create a report list
report = []

# 1. Find high-confidence false positives (likely annotation errors)
fp_view = dataset.filter_labels(
    "predictions",
    (F("confidence") > 0.85) & (F("eval") == "fp")
)

for sample in fp_view:
    report.append({
        "filepath": sample.filepath,
        "problems": ["High-confidence false positive - possible missing ground truth annotation"],
        "sample_id": sample.id
    })

# 2. Find samples with high mistakenness scores
mistake_view = dataset.filter_labels("ground_truth", F("mistakenness") > 0.95)

for sample in mistake_view:
    report.append({
        "filepath": sample.filepath,
        "problems": ["High mistakenness score - likely annotation error"],
        "sample_id": sample.id
    })

# 3. Find possible missing annotations
missing_view = dataset.match(F("possible_missing") > 0)

for sample in missing_view:
    report.append({
        "filepath": sample.filepath,
        "problems": ["Possible missing annotations detected by model"],
        "sample_id": sample.id
    })

# 4. Find possible spurious annotations
spurious_view = dataset.match(F("possible_spurious") > 0)

for sample in spurious_view:
    report.append({
        "filepath": sample.filepath,
        "problems": ["Possible spurious/incorrect annotations"],
        "sample_id": sample.id
    })
    
print(f"Found {len(report)} problems needing review")
print('Note that some samples (images) may have multiple problems.')

# print(f"Report generated with {len(report)} samples needing review")
# print("Report saved to: reannotation_report.json")

# # Tag samples in FiftyOne for easy filtering
# sample_ids = [item["sample_id"] for item in report]
# view = dataset.select(sample_ids)
# view.tag_samples("needs_reannotation")

# print(f"Tagged {len(sample_ids)} samples with 'needs_reannotation' tag")

Evaluating detections...
 100% |█████████████████| 151/151 [977.7ms elapsed, 0s remaining, 158.3 samples/s]      
Performing IoU sweep...
 100% |█████████████████| 151/151 [660.4ms elapsed, 0s remaining, 228.7 samples/s]      
Evaluating detections...
 100% |█████████████████| 151/151 [1.1s elapsed, 0s remaining, 145.8 samples/s]         
Computing mistakenness...
 100% |█████████████████| 151/151 [961.3ms elapsed, 0s remaining, 160.7 samples/s]      
Mistakenness computation complete
Report generated with 45 samples needing review
Report saved to: reannotation_report.json
Tagged 45 samples with 'needs_reannotation' tag


In [8]:
len(report)

45

## Create a YOLO dataset which can be imported by the DigitalSreeni Image Annotator

It looks like the image annotator imports only the train split of a YOLO dataset.

So the following code puts flattens the dataset to just 2 data folders ('images' and 'labels').

The split dataset needs to be reconstructed after reannotation.

In [ ]:
# Create a YOLO dataset of the problematic samples for reannotation

for i in ['images', 'labels']:
    for j in ['train', 'val']:
        os.makedirs(f'{OUTPUT_DIR}/{i}/{j}', exist_ok=True)

i = 0
for image in report:
    i += 1
    image_filepath = image['filepath']
    data_split = image_filepath.split('/')[-2]
    label_filepath = image_filepath.replace('/images/', '/labels/').replace('.jpg', '.txt') 
    # print(i, data_split, image_filepath)
    shutil.copy2(image_filepath, f'{OUTPUT_DIR}/images/{data_split}')
    shutil.copy2(label_filepath, f'{OUTPUT_DIR}/labels/{data_split}')
    
# Add a YAML file
s = f'''
names:
- healthy
- damaged
- dead
- vcut
path: {OUTPUT_DIR}
train: images/train
val: images/val
'''
with open(f'{OUTPUT_DIR}/dataset.yaml', 'w') as f:
    f.write(s) 
    
# Create "problematic_images_report.csv" and save it in OUTPUT_DIR
def filepath2filename(filepath):
    return filepath.split('/')[-1]

def problems2problem_str(problems):
    return ', '.join(problems)

df = pd.DataFrame(report)
df['filename'] = df['filepath'].apply(filepath2filename)
df['problems_str'] = df['problems'].apply(problems2problem_str)
df.drop(columns=['filepath', 'problems', 'sample_id'], inplace=True )
df.sort_values(by=['filename', 'problems_str'], inplace=True)

# Group by col1 and concatenate strings in col2 with commas
new_df = df.groupby('filename')['problems_str'].apply(lambda x: '|'.join(x)).reset_index()

new_df.to_csv(f'{OUTPUT_DIR}/problematic_images_report.csv', index=False)
print(f"Saved report to {OUTPUT_DIR}/problematic_images_report.csv")

In [ ]:
print('Reannotation dataset is at {OUTPUT_DIR}')
print('FINISHED')

FINISHED: reannotation dataset at /home/aubrey/Desktop/reannotation_dataset


## Next steps
- Launch digitalsreeni image annotator: `am_image_annotator.sh`
- Import the new reannotation dataset 
- Fix problems identified in problematic_images_report.csv
- Add new annotations to the old ones
- Retrain